In [1]:
import dataset
import dtw_recognition
import numpy as np

In [2]:
np.random.seed(42)

# DTW

In [3]:
config = {
    "data_dir": ["../data/train/19021396_PhamThanhVinh"],
    "sr": 22050,
    "n_mfcc": 13,
    "hop_length": 256,
    "n_fft": 512,
    "delta_width": 5,
    "n_sample_per_word": 3,
}
df = dataset.get_dataset_df(config["data_dir"], config["sr"], config["n_mfcc"], config["hop_length"], config["n_fft"],
                   config["delta_width"])
template_df = dataset.get_template_df(df, config["n_sample_per_word"])

In [4]:
df

,label,mfcc_features
0,sil,"[[0.0, 0.36298612, 1.0, 0.67046523, 1.0, 0.109..."
1,1,"[[0.0, 0.075219214, 0.5102628, 0.37972867, 0.6..."
2,tram,"[[0.29391974, 0.69984466, 0.7457707, 0.3506259..."
3,4,"[[0.9442609, 0.0, 0.50626904, 0.52150154, 0.72..."
4,muoi,"[[1.0, 0.54144883, 0.37757295, 0.39642286, 0.0..."
...,...,...
1775,tram,"[[0.26855448, 0.39782253, 0.06277703, 0.511701..."
1776,8,"[[0.006426136, 0.03151538, 1.0, 0.63716173, 0...."
1777,muoi,"[[1.0, 0.16773497, 1.0, 0.20140655, 0.2975387,..."
1778,tu,"[[0.3128125, 0.8595457, 0.54761374, 0.0, 0.365..."


In [5]:
list(df["label"].unique())

['sil',
 '1',
 'tram',
 '4',
 'muoi',
 '9',
 'trieu',
 '3',
 '7',
 '8',
 'nghin',
 '6',
 '5',
 'lam',
 '2',
 'tu',
 '0',
 'mot',
 'linh',
 'm1']

In [13]:
# avg = {}
# min = {}
# max = {}
#
# syllables = pd.DataFrame(columns=["label", "n_syllables"])
# syllables["label"] = df["label"].unique()
# syllable_list = []
#
# for label in df["label"].unique():
#     dfi = df[df["label"] == label]
#     avg_i = 0
#     min_i = np.inf
#     max_i = 0
#     for i in range(len(dfi)):
#         a = dfi["mfcc_features"].iloc[i].shape[0]
#         avg_i += a
#         if a < min_i:
#             min_i = a
#         if a > max_i:
#             max_i = a
#     avg[label] = int(avg_i/len(dfi))
#     min[label] = min_i
#     max[label] = max_i
#
# rate = 0.6
# for i in range(len(syllables)):
#     s_i = int(min[syllables["label"].iloc[i]]*rate)
#     if s_i >= 6:
#         s_i -= 3
#     syllable_list.append(s_i)
# syllables["n_syllables"] = syllable_list
#
# syllables.to_csv("../data/syllables.csv", sep="\t", header=None, index=False)

In [14]:
template_df

,label,mfcc_features
0,sil,"[[0.0, 0.0, 1.0, 0.21681933, 1.0, 1.0, 0.63988..."
1,sil,"[[0.98916966, 0.9665106, 0.6104375, 0.2689894,..."
2,sil,"[[1.0, 0.0, 0.8766952, 0.0, 0.59409726, 0.1476..."
3,1,"[[0.0, 0.0, 0.7700621, 0.6620277, 0.8244615, 0..."
4,1,"[[1.0, 0.1181881, 0.7809504, 0.49549296, 0.994..."
5,1,"[[0.0, 0.0, 0.18497944, 0.17161536, 0.72778875..."
6,tram,"[[0.0, 0.44544995, 0.97258705, 0.5779728, 0.94..."
7,tram,"[[0.123844154, 0.5532935, 0.9188345, 0.3868457..."
8,tram,"[[0.03165088, 0.70897883, 0.58971876, 0.478225..."
9,4,"[[0.07372889, 0.57898265, 0.6029618, 0.7811839..."


In [15]:
new_template_df = dtw_recognition.reduce_models(template_df, "../data/syllables.csv")

In [16]:
choice_indexes = np.random.choice(len(df), 100, replace=False)
test_mfcc_features = list(df.loc[choice_indexes, "mfcc_features"])

In [17]:
df.iloc[choice_indexes]

,label,mfcc_features
1192,sil,"[[1.0, 0.23899186, 0.861323, 0.0, 0.35400915, ..."
426,nghin,"[[0.9680871, 0.47679815, 0.89284927, 0.0, 0.52..."
1475,2,"[[0.2106057, 0.0, 0.36059535, 0.72391516, 1.0,..."
765,3,"[[0.05818045, 0.0, 1.0, 0.9120276, 0.72712815,..."
485,tram,"[[0.18089612, 0.568977, 0.7462516, 0.4967905, ..."
...,...,...
598,trieu,"[[0.35030463, 0.44171703, 0.77174234, 0.395985..."
939,muoi,"[[1.0, 0.5335298, 0.6074466, 0.13665825, 0.0, ..."
367,sil,"[[0.38811898, 0.23959908, 0.90351385, 0.273567..."
162,7,"[[0.13198762, 0.5267974, 1.0, 0.5493203, 0.526..."


In [18]:
# preds = dtw_recognition.predict(template_df, test_mfcc_features, 2)
# preds = dtw_recognition.predict(new_template_df, test_mfcc_features, 2)

In [19]:
print(f'Accuracy: {np.sum(preds == df.loc[choice_indexes, "label"])/len(preds)}')

NameError: name 'preds' is not defined

# HMM

In [23]:
np.array(df.loc[choice_indexes, "label"]) == "sil"

array([ True, False, False, False, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False,  True, False,
        True, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False,  True, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True,  True, False, False,  True, False,
        True])